In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install pyTelegramBotApi
!pip install transformers
import telebot
from typing import Tuple, List
from sklearn.model_selection import train_test_split
from transformers import (AutoTokenizer, TextDataset, DataCollatorForLanguageModeling,
                          BertForMaskedLM,
                          Trainer, TrainingArguments,
                          TextGenerationPipeline, pipeline)
from torch.cuda import empty_cache
import os
import time
import copy
import torch
import re
from torch.nn import functional as F

eval_batch_size : int = 16
model_name       : str = "sberbank-ai/sbert_large_nlu_ru"
split_token      : str = "<|endoftext|>"
output_dir = "/content/drive/MyDrive/Sbert model/"
#print("Введите ключ к боту")
#botkey=input()
botkey="5307947497:AAHsHOMXbRFLM28h0-SNCNj0M8rAbesj4YM"
bot = telebot.TeleBot(botkey)

def generateStr(start, size):
    generated = start
    for i in range(size):
        print(generated)
        generated = generated + " " + tokenizer.mask_token
        input = tokenizer.encode_plus(generated, return_tensors = "pt")
        mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)
        output = model(**input)
        logits = output.logits
        softmax = F.softmax(logits, dim = -1)
        mask_word = softmax[0, mask_index, :]
        top_10 = torch.topk(mask_word, 10, dim = 1)[1][0]
        for token in top_10:
            print(tokenizer.decode([token]))
        for token in top_10:
            word = tokenizer.decode([token])
            if(word != "[CLS]"):
                generated = generated.replace(tokenizer.mask_token, word)
                print(generated)
                break
        print("\n")
    return generated

def predictStr(start):
    words = start.replace("\n","").split(" ")
    length = range(len(words))
    new_words = []
    print(start)
    print("->\n")
    for i in length:
        expected = words[i] 
        if not (expected.find(".ru") == -1 and expected.find(".com") == -1 and expected.find("@") == -1 and expected.find("+7") == -1):
            new_words.append(expected)
            continue
        if expected == "" or expected == " ":
            continue
        text = " ".join(words[j] if j != i else tokenizer.mask_token for j in length)
        input = tokenizer.encode_plus(text, return_tensors = "pt")
        mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)
        output = model(**input)
        logits = output.logits
        softmax = F.softmax(logits, dim = -1)
        mask_word = softmax[0, mask_index, :]
        #token = torch.topk(mask_word, 1, dim = 1)[1][0]
        #word = tokenizer.decode(token)
        top_10 = torch.topk(mask_word, 10, dim = 1)[1][0]
        for token in top_10:
            word = tokenizer.decode([token])
            new_sentence = text.replace(tokenizer.mask_token, word)
            print(new_sentence)
        print("\n")
        if(tokenizer.decode([top_10[0]]).lower() == expected.lower()):
            new_words.append("*" + expected + "*")
        else:
            new_words.append(expected)

    return " ".join(new_words)


def predict(start, expect) -> bool:
    eval_model = TextGenerationPipeline(model=model, 
                                    tokenizer=tokenizer,
                                    batch_size=eval_batch_size,
                                    device=0,
                                    max_length=50)
    predictions = eval_model(start)
    print(expect, predictions[0]["generated_text"].split(" ")[len(start)], expect == predictions[0]["generated_text"].split(" ")[len(start)])
    try:
        if expect == predictions[0]["generated_text"].split(" ")[len(start)]:
            return True
    except Exception as e:
        return False
    return False

def load_model(path : str = model_name, resize : bool = True) -> Tuple[AutoTokenizer, BertForMaskedLM]:
    tokenizer = AutoTokenizer.from_pretrained(model_name, eos_token=split_token, pad_token=split_token)
    PATH = 'config.json'
    model = BertForMaskedLM.from_pretrained(path, local_files_only=False)
    if resize:
        model.resize_token_embeddings(len(tokenizer))
    return tokenizer, model

@bot.message_handler(commands=["start"])
def start(m, res=False):
    bot.send_message(m.chat.id, 'Я на связи. Напиши мне что-нибудь )')

@bot.message_handler(content_types=["text"])
def handle_text(message):
    bot.send_message(message.chat.id, 'Вы написали: ' + message.text)
    mes = message.text.replace("\n"," ")
    #bot.send_message(message.chat.id, 'Мы написали: ' + predictStr(mes))
    bot.send_message(message.chat.id, 'Мы написали: ' + generateStr(mes, 15))

@bot.message_handler(content_types=["document"])
def handle_document(message):
    if message.document.mime_type != 'text/plain':
        bot.send_message(message.chat.id, 'Вы отправили файл не в формате txt')
        return

    file_path = bot.get_file(message.document.file_id)
    downloaded_file = bot.download_file(file_path.file_path)
    with open(message.document.file_name, 'wb') as new_file:
        new_file.write(downloaded_file)

    new_file = open(message.document.file_name, 'r', encoding = 'utf-8')
    downloaded_file = new_file.read().lower()
    bot.send_message(message.chat.id, 'Проверка структуры')
    if(downloaded_file.find("— это") == -1):
        bot.send_message(message.chat.id, 'Отсутствует основные положения')
    if(downloaded_file.find("компани") == -1):
        bot.send_message(message.chat.id, 'Отсутствует информация о компании')
    if(downloaded_file.find("третьим лицам") == -1 and downloaded_file.find("третьи лица") == -1):
        bot.send_message(message.chat.id, 'Отсутствует информация о распространении')
    if(downloaded_file.find("срок действия") == -1 and downloaded_file.find(" хранени") == -1):
        bot.send_message(message.chat.id, 'Отсутствует информация о хранении')
    if(downloaded_file.find("обратная связь") == -1 and downloaded_file.find("контакт") == -1):
        bot.send_message(message.chat.id, 'Отсутствует информация об обратной связи')

    new_file.seek(0)
    with open(message.document.file_name.replace(".txt","") + "_edit.txt", 'w', encoding = 'utf-8') as result:

        doclist = [line.replace("\n"," ") for line in new_file]
        docstr = ''. join(doclist)
        sentences = re.split(r'[.!?;]\s', docstr)

        for sentence in sentences:
            result.write(predictStr(sentence) + ".\n")
        
    new_file.close()
    with open(message.document.file_name.replace(".txt","") + "_edit.txt", 'rb') as new_file:
        bot.send_document(message.chat.id, new_file)
    
    os.remove(message.document.file_name)
    os.remove(message.document.file_name.replace(".txt","") + "_edit.txt")
    bot.send_message(message.chat.id, 'Обработано')

tokenizer, model = load_model("/content/drive/MyDrive/Sbert model/checkpoint-10000", resize=False)

while True:
    #try:
        bot.polling(none_stop=True, interval=0)

    #except Exception as e:
        #print(e)  # или просто print(e) если у вас логгера нет,
        #time.sleep(1)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 157 kB 5.1 MB/s 
  Created wheel for pyTelegramBotApi: filename=pyTelegramBotAPI-4.5.1-py3-none-any.whl size=138116 sha256=7e46355f6c3009cd97e9d106129ac7308b9386ea38b2f58f68447532d5a22519
  Stored in directory: /root/.cache/pip/wheels/9a/90/69/89e4e68c819633d933753827fd4f3a22987e162b7404b092b5
Successfully built pyTelegramBotApi
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.1 MB/s 
     |████████████████████████████████| 86 kB 6.6 MB/s 
     |████████████████████████████████| 6.6 MB 47.6 MB/s 
     |████████████████████████████████| 596 kB 38.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Downloading:   0%|          | 0.00/323 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/655 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.70M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Политика использования файлов Cookie Согласие Что такое файлы cookies
->

Насчет использования файлов Cookie Согласие Что такое файлы cookies
Относительно использования файлов Cookie Согласие Что такое файлы cookies
После использования файлов Cookie Согласие Что такое файлы cookies
Для использования файлов Cookie Согласие Что такое файлы cookies
Касательно использования файлов Cookie Согласие Что такое файлы cookies
Пример использования файлов Cookie Согласие Что такое файлы cookies
[CLS] использования файлов Cookie Согласие Что такое файлы cookies
Без использования файлов Cookie Согласие Что такое файлы cookies
для использования файлов Cookie Согласие Что такое файлы cookies
Опыт использования файлов Cookie Согласие Что такое файлы cookies


Политика [CLS] файлов Cookie Согласие Что такое файлы cookies
Политика , файлов Cookie Согласие Что такое файлы cookies
Политика на файлов Cookie Согласие Что такое файлы cookies
Политика – файлов Cookie Согласие Что такое файлы cookies
Политика е